In [1]:
# i know i repeated myself
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df = pd.read_csv('../DSI-CHI-1/lessons/week-06/3.1-random-forests-boosting/assets/datasets/car.csv')

In [3]:
df.head(2)

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc


Prepare the data

Load the data into a pandas dataframe

Encode the categorical features properly: 
define a map that preserves the scale (assigning smaller numbers to words indicating smaller quantities)

Separate features from target into X and y

In [4]:
y = LabelEncoder().fit_transform(df['acceptability'])
X = pd.get_dummies(df.drop('acceptability', axis=1))

In [5]:
y

array([2, 2, 2, ..., 2, 1, 3])

In [6]:
# dropped target from df and made dummie_var for our columns and expanded options in those columns to be variables
X.head(2)

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


Useful preparation

Separate X and y between a train and test set, using 30% test set, random state = 42
make sure that the data is shuffled and stratified

Define a function called evaluate_model that trains the model on the train set tests it on the test and then calculates:

accuracy score

confusion matrix

classification report

Initialize a global dictionary to store the various models for later retrieval

In [29]:
# setting up train and test data for X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(1209, 21) (1209,)
(519, 21) (519,)


In [30]:
# data is shuffled
cv = KFold(len(df),n_folds=3, shuffle=True, random_state=42)

In [31]:
cv

sklearn.cross_validation.KFold(n=1728, n_folds=3, shuffle=True, random_state=42)

In [32]:
KFold?

In [33]:
StratifiedKFold?

In [34]:
# stratified and shuffled these are the ones for train and test set
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=42)

In [35]:
cv

sklearn.cross_validation.StratifiedKFold(labels=[2 2 2 ..., 2 1 3], n_folds=3, shuffle=True, random_state=42)

In [36]:
def evaluate_model(X_train, X_test, 
                   y_train, y_test,
                  model):
    model.fit(x_train, y_train)
    print model.score(x_test, y_test)
    predictions = model.predict(X_test)
    # find confuse, class

In [37]:
# cross val list of scores is accuracy of each folds. avg cv score of all folds
# def score(model, name):
#     s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
#     print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [38]:
dt = DecisionTreeClassifier(class_weight='balanced')
# takes model i set up and takes x and y's and runs each fold through model
# we want to see more though
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)

In [39]:
print "{} Score:\t{:0.3} ± {:0.3}".format("Decision Tree", s.mean().round(3), s.std().round(3))

Decision Tree Score:	0.953 ± 0.011


In [40]:
bdt = BaggingClassifier(DecisionTreeClassifier())

In [41]:
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

In [42]:
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)

In [59]:
# score(dt, "Decision Tree")

In [56]:
# score(bdt, "Bagging DT")

In [57]:
# score(rf, "Random Forest")

In [58]:
#score(et, "Extra Trees")

In [47]:
# want to build a global dict to store all my models for future use
global_models = {
            'bdt': BaggingClassifier(DecisionTreeClassifier()), 
            'rf': RandomForestClassifier(class_weight='balanced', n_jobs=-1),
            'et': ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)
}    

In [53]:
results = {}

def evaluate_model(X_train, X_test, 
                   y_train, y_test,
                  model):
    pick_a_model = global_models[model]
    pick_a_model.fit(X_train, y_train)
    print 'train model score:', pick_a_model.score(X_train, y_train)
    predictions = pick_a_model.predict(X_test)
    print '\nconfusion matrix: \n\n', confusion_matrix(y_test, predictions)
    print '\nclassification report: \n\n', classification_report(y_test, predictions)
    results.update({model: pick_a_model.score(X_test, y_test)})

In [54]:
for key in global_models:
    evaluate_model(X_train, X_test, y_train, y_test, key)

train model score: 0.998345740281

confusion matrix: 

[[110   0   1   0]
 [  2  14   0   3]
 [  9   0 365   0]
 [  0   1   0  14]]

classification report: 

             precision    recall  f1-score   support

          0       0.91      0.99      0.95       111
          1       0.93      0.74      0.82        19
          2       1.00      0.98      0.99       374
          3       0.82      0.93      0.87        15

avg / total       0.97      0.97      0.97       519

train model score: 1.0

confusion matrix: 

[[107   1   3   0]
 [  4  11   0   4]
 [ 17   0 357   0]
 [  0   0   0  15]]

classification report: 

             precision    recall  f1-score   support

          0       0.84      0.96      0.90       111
          1       0.92      0.58      0.71        19
          2       0.99      0.95      0.97       374
          3       0.79      1.00      0.88        15

avg / total       0.95      0.94      0.94       519

train model score: 1.0

confusion matrix: 

[[107   0

In [55]:
results

{'bdt': 0.96917148362235073,
 'et': 0.96146435452793833,
 'rf': 0.94412331406551064}

In [ ]:
counter = 0
set_of_scores = []

for model in evaluate_model:
#     decision_tree = DecisionTreeRegressor()
#     x_train = x.as_matrix()[training_set]
#     x_test = x.as_matrix()[test_set]
#     y_train = y.as_matrix()[training_set]
#     y_test = y.as_matrix()[test_set]
#     model_score = fit_and_evaluate_model(x_train, x_test, 
#                                          y_train, y_test,
#                                         decision_tree)
#     print 'for fold %s, the score is: %s' % (counter, model_score)
#     set_of_scores.append(model_score)
#     counter += 1
    
# print 'the avg score is: %s' % np.mean(set_of_scores)

In [ ]:
all_the_scores = []

for i in all_the_scores:
    for model in pick_a_model():
        
        print evaluate_model() 

In [ ]:
for i in evaluate_model:
    for model in pick_a_model():
        print 